In [8]:
import os
import shutil
from shutil import copyfile
train_filenames = os.listdir('train')
train_cat = filter(lambda x:x[:3] == 'cat', train_filenames)
train_dog = filter(lambda x:x[:3] == 'dog', train_filenames)
def rmrf_mkdir(dirname):
    if os.path.exists(dirname):
        shutil.rmtree(dirname)
    os.mkdir(dirname)
rmrf_mkdir('train2')
os.mkdir('train2/cat')
os.mkdir('train2/dog')

for filename in train_cat:
    copyfile('train/'+filename, 'train2/cat/'+filename)
for filename in train_dog:
    copyfile('train/'+filename, 'train2/dog/'+filename)

In [13]:
rmrf_mkdir('test2')
os.mkdir('test2/test')
test_filenames = os.listdir('test')

for filename in test_filenames:
    os.symlink('test/'+filename, 'test2/test/'+filename)

## 导出 fine-tune 的特征向量

In [2]:
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
import h5py

Using TensorFlow backend.


In [3]:
def export_gap_fine_tune(model,save_name,image_size, lambda_func=None):
    width = image_size[0]
    height = image_size[1]
    input_tensor = Input((height, width, 3))
    x = input_tensor
    if lambda_func:
        x = Lambda(lambda_func)(x)
    
    gen = ImageDataGenerator()
    train_generator = gen.flow_from_directory("train2", image_size, shuffle=False, 
                                              batch_size=16)
    test_generator = gen.flow_from_directory("test2", image_size, shuffle=False, 
                                             batch_size=16, class_mode=None)
    train_steps = train_generator.samples / 16
    test_steps = test_generator.samples / 16
    train = model.predict_generator(train_generator, train_steps)
    test = model.predict_generator(test_generator, test_steps)
    with h5py.File(save_name) as h:
        h.create_dataset("train", data=train)
        h.create_dataset("test", data=test)
        h.create_dataset("label", data=train_generator.classes)

In [3]:
vgg16_base_model = load_model("models/vgg16_last_nofreeze.h5")
vgg16_base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [4]:
vgg16_model = Model(inputs=vgg16_base_model.input,
                    outputs=vgg16_base_model.get_layer('global_average_pooling2d_2').output) 

In [10]:
export_gap_fine_tune(vgg16_model,"gap_vgg16_model.h5",(224,224))

Found 25000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.


In [11]:
resnet50_base_model = load_model("models/resnet_50_last_layer_able")
resnet50_base_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_3 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
zero_padding2d_3 (ZeroPadding2D) (None, 230, 230, 3)   0           input_3[0][0]                    
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, 112, 112, 64)  9472        zero_padding2d_3[0][0]           
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, 112, 112, 64)  256         conv1[0][0]                      
___________________________________________________________________________________________

In [12]:
resnet50_model = Model(inputs=resnet50_base_model.input,
                    outputs=resnet50_base_model.get_layer('global_average_pooling2d_3').output) 

In [13]:
export_gap_fine_tune(resnet50_model,"gap_resnet50_model.h5",(224,224))

Found 25000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.


In [3]:
inceptionv3_model = InceptionV3(input_tensor=Input((299, 299, 3)),include_top=False,pooling='avg')

In [5]:
export_gap_fine_tune(inceptionv3_model,"gap_inception3_model.h5",(299,299),inception_v3.preprocess_input)

Found 25000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.


In [7]:
xception_model = Xception(input_tensor=Input((299,299,3)), include_top=False, pooling='avg')
export_gap_fine_tune(xception_model,'gap_xception_model.h5',(299,299), xception.preprocess_input)

Found 25000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.


In [3]:
inception_v3_ft_base_model = load_model('models/inception3_fine_tune.h5')
inception_v3_ft_base_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 299, 299, 3)   0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 149, 149, 32)  864         input_1[0][0]                    
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, 149, 149, 32)  96          conv2d_1[0][0]                   
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 149, 149, 32)  0           batch_normalization_1[0][0]      
___________________________________________________________________________________________

In [5]:
inception_v3_model = Model(inputs=inception_v3_ft_base_model.input,
                    outputs=inception_v3_ft_base_model.get_layer('global_average_pooling2d_1').output)
export_gap_fine_tune(inception_v3_model,"gap_inception_tf_model.h5",(299,299))

Found 25000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.


In [4]:
xception_ft_base_model = load_model('models/xception_ft_model.h5')
xception_ft_base_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 299, 299, 3)   0                                            
____________________________________________________________________________________________________
block1_conv1 (Conv2D)            (None, 149, 149, 32)  864         input_1[0][0]                    
____________________________________________________________________________________________________
block1_conv1_bn (BatchNormalizat (None, 149, 149, 32)  128         block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_conv1_act (Activation)    (None, 149, 149, 32)  0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [5]:
xception_model = Model(inputs=xception_ft_base_model.input, 
                       outputs=xception_ft_base_model.get_layer('global_average_pooling2d_1').output)

export_gap_fine_tune(xception_model, 'xception_tf_model.h5',(299,299))

Found 25000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.


In [1]:
import h5py
import numpy as np
from sklearn.utils import shuffle
np.random.seed(2017)
X_train = []
X_test = []
for filename in ["gap_resnet50_model.h5", "gap_vgg16_model.h5","gap_inception_tf_model.h5","xception_tf_model.h5"]:
    with h5py.File(filename, 'r') as h:
        X_train.append(np.array(h['train']))
        X_test.append(np.array(h['test']))
        y_train = np.array(h['label'])
X_train = np.concatenate(X_train, axis=1)
X_test = np.concatenate(X_test, axis=1)
X_train, y_train = shuffle(X_train, y_train)

In [2]:
from keras.models import *
from keras.layers import *
np.random.seed(2017)
input_tensor = Input(X_train.shape[1:])
x = Dropout(0.5)(input_tensor)
x = Dense(1, activation='sigmoid')(x)
model = Model(input_tensor, x)
model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])

Using TensorFlow backend.


In [3]:
model.fit(X_train, y_train, batch_size=128, epochs=20, validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/20
20000/20000 [==============================] - 1s - loss: 0.2402 - acc: 0.9123 - val_loss: 0.0302 - val_acc: 0.9890
Epoch 2/20
20000/20000 [==============================] - 0s - loss: 0.0437 - acc: 0.9845 - val_loss: 0.0234 - val_acc: 0.9932
Epoch 3/20
20000/20000 [==============================] - 0s - loss: 0.0270 - acc: 0.9906 - val_loss: 0.0244 - val_acc: 0.9922
Epoch 4/20
20000/20000 [==============================] - 0s - loss: 0.0230 - acc: 0.9917 - val_loss: 0.0267 - val_acc: 0.9916
Epoch 5/20
20000/20000 [==============================] - 0s - loss: 0.0196 - acc: 0.9933 - val_loss: 0.0199 - val_acc: 0.9936
Epoch 6/20
20000/20000 [==============================] - 0s - loss: 0.0173 - acc: 0.9939 - val_loss: 0.0194 - val_acc: 0.9942
Epoch 7/20
20000/20000 [==============================] - 0s - loss: 0.0173 - acc: 0.9939 - val_loss: 0.0197 - val_acc: 0.9934
Epoch 8/20
20000/20000 [==============================] - 0s -

In [4]:
y_pred = model.predict(X_test, verbose=1)
y_pred = y_pred.clip(min=0.005, max=0.995)
import pandas as pd
from keras.preprocessing.image import *
df = pd.read_csv("sample_submission.csv")
gen = ImageDataGenerator()
test_generator = gen.flow_from_directory("test2", (224, 224), shuffle=False, 
                                         batch_size=16, class_mode=None)
for i, fname in enumerate(test_generator.filenames):
    index = int(fname[fname.rfind('/')+1:fname.rfind('.')])
    df.set_value(index-1, 'label', y_pred[i])
df.to_csv('merge_4_pred.csv', index=None)
df.head(10)

11488/12500 [==========================>...] - ETA: 0sFound 12500 images belonging to 1 classes.


,id,label
0,1,0.995
1,2,0.995
2,3,0.995
3,4,0.995
4,5,0.005
5,6,0.005
6,7,0.005
7,8,0.005
8,9,0.005
9,10,0.005


In [19]:
import h5py
import numpy as np
from sklearn.utils import shuffle
np.random.seed(2017)
X_train = []
X_test = []
for filename in ["gap_inception3_model.h5","gap_xception_model.h5"]:
    with h5py.File(filename, 'r') as h:
        X_train.append(np.array(h['train']))
        X_test.append(np.array(h['test']))
        y_train = np.array(h['label'])
X_train = np.concatenate(X_train, axis=1)
X_test = np.concatenate(X_test, axis=1)
X_train, y_train = shuffle(X_train, y_train)

In [20]:
from keras.models import *
from keras.layers import *
np.random.seed(2017)
input_tensor = Input(X_train.shape[1:])
x = Dropout(0.5)(input_tensor)
x = Dense(1, activation='sigmoid')(x)
model = Model(input_tensor, x)
model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [22]:
model.fit(X_train, y_train, batch_size=128, epochs=8, validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/8
20000/20000 [==============================] - 0s - loss: 2.6915 - acc: 0.6186 - val_loss: 1.6367 - val_acc: 0.6688
Epoch 2/8
20000/20000 [==============================] - 0s - loss: 2.6756 - acc: 0.6174 - val_loss: 1.3872 - val_acc: 0.6978
Epoch 3/8
20000/20000 [==============================] - 0s - loss: 2.5117 - acc: 0.6211 - val_loss: 1.1628 - val_acc: 0.7070
Epoch 4/8
20000/20000 [==============================] - 0s - loss: 2.4471 - acc: 0.6214 - val_loss: 1.8684 - val_acc: 0.6456
Epoch 5/8
20000/20000 [==============================] - 0s - loss: 2.4665 - acc: 0.6191 - val_loss: 5.6048 - val_acc: 0.5032
Epoch 6/8
20000/20000 [==============================] - 0s - loss: 2.5473 - acc: 0.6126 - val_loss: 2.7849 - val_acc: 0.5676
Epoch 7/8
20000/20000 [==============================] - 0s - loss: 2.4790 - acc: 0.6168 - val_loss: 3.2400 - val_acc: 0.5558
Epoch 8/8
20000/20000 [==============================] - 0s - loss: 2

In [13]:
y_pred = model.predict(X_test, verbose=1)
y_pred = y_pred.clip(min=0.005, max=0.995)
import pandas as pd
from keras.preprocessing.image import *
df = pd.read_csv("sample_submission.csv")
gen = ImageDataGenerator()
test_generator = gen.flow_from_directory("test2", (224, 224), shuffle=False, 
                                         batch_size=16, class_mode=None)
for i, fname in enumerate(test_generator.filenames):
    index = int(fname[fname.rfind('/')+1:fname.rfind('.')])
    df.set_value(index-1, 'label', y_pred[i])
df.to_csv('merge_4_pred.csv', index=None)
df.head(10)

11584/12500 [==========================>...] - ETA: 0sFound 12500 images belonging to 1 classes.


,id,label
0,1,0.995
1,2,0.995
2,3,0.995
3,4,0.995
4,5,0.005
5,6,0.005
6,7,0.005
7,8,0.005
8,9,0.005
9,10,0.005
